In [2]:
import pandas as pd
import numpy as np

In [3]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
df_train = pd.read_csv('train.csv', index_col=0)

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
X = df_train.drop("SalePrice", axis=1)
y = df_train["SalePrice"]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [ ]:
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_test = X_test[my_cols].copy()

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [13]:
prepared_data = preprocessor.fit_transform(X_train)

In [14]:
prepared_data

array([[2.000e+01, 7.000e+01, 8.400e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [6.000e+01, 5.900e+01, 7.837e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [3.000e+01, 6.700e+01, 8.777e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [2.000e+01, 6.000e+01, 8.172e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [5.000e+01, 5.500e+01, 7.642e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.200e+02, 5.300e+01, 3.684e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00]])

In [15]:
from sklearn.ensemble import RandomForestRegressor

rnd = RandomForestRegressor(n_estimators=100)

In [16]:
from sklearn.metrics import mean_squared_error as mse

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', rnd)
                             ])

In [17]:
my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCo

In [18]:
preds = my_pipeline.predict(X_test)

In [19]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, preds)

17815.58366438356

In [20]:
from sklearn.metrics import mean_squared_error as mse
import numpy as np


np.sqrt(mse(y_test, preds))

29210.756615726674

In [21]:
df_test = pd.read_csv('train.csv', index_col=0)

In [22]:
df_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [23]:
X = df_test.drop("SalePrice", axis=1)
y = df_test["SalePrice"]

preds_test = my_pipeline.predict(X)

/home/rockstar/ml_env/lib/python3.6/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [24]:
val = mse(preds_test, y)

In [25]:
val

269531701.61918896

In [26]:
np.sqrt(val)

16417.420674977813

In [27]:
preds_test

array([204231.5 , 179183.  , 217763.5 , ..., 255635.14, 139758.5 ,
       149292.25])

In [28]:
df_sub = pd.read_csv("sample_submission.csv")

In [29]:
df_sub

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [30]:
len(preds_test)

1460

In [31]:
len(df_sub)

1459

In [32]:
df_sub = df_sub.loc[1:]

In [33]:
df_sub

,Id,SalePrice
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
5,1466,177150.989247
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [34]:
len(df_sub)

1458

In [35]:
len(preds_test)

1460

In [36]:
my_pipeline.score(X_test, y_test)

0.8887572341755552